<a href="https://colab.research.google.com/github/lilswapnil/book-recommender/blob/main/notebook/vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Vector Search

In [ ]:
%%writefile requirements.txt
python-dotenv>=1.0
kaggle
kagglehub
numpy
pandas
matplotlib
seaborn

# Prevent Chroma import crash
pydantic<2.12
pydantic-settings<2.12

torch
transformers

# Keep your current working LangChain/Chroma combo stable
chromadb==1.5.0
langchain==1.2.10
langchain-chroma==1.1.0
langchain-community==0.4.1
langchain-core==1.2.12
langchain-openai==1.1.9
langchain-text-splitters==1.1.0

Overwriting requirements.txt


In [ ]:
%pip install -r requirements.txt

In [2]:
from google.colab import userdata

HF_TOKEN = userdata.get("HUGGINGFACEHUB_API_TOKEN")
OPENAI_KEY = userdata.get("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Define the path where you saved the file in Google Drive
drive_path = '/content/drive/MyDrive/books.csv'

# Load the CSV file into a pandas DataFrame
loaded_books_df = pd.read_csv(drive_path)

# Display the first few rows to verify
print(f"Successfully loaded 'books.csv' from Google Drive. Shape: {loaded_books_df.shape}")
display(loaded_books_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully loaded 'books.csv' from Google Drive. Shape: (75750, 8)


,author,desc,genre,isbn,rating,reviews,title,tag
0,Laurence M. Hauptman,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",002914180X,3.52,5,Between Two Fires: American Indians in the Civ...,002914180X Reveals that several hundred thousa...
1,"Charlotte Fiell,Emmanuelle Dirix",Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",1906863482,4.51,6,Fashion Sourcebook 1920s,1906863482 Fashion Sourcebook - 1920s is the f...
2,Andy Anderson,The seminal history and analysis of the Hungar...,"Politics,History",948984147,4.15,2,Hungary 56,948984147 The seminal history and analysis of ...
3,Carlotta R. Anderson,"""All-American Anarchist"" chronicles the life a...","Labor,History",814327079,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,"814327079 ""All-American Anarchist"" chronicles ..."
4,Jeffrey Pfeffer,Why is common sense so uncommon when it comes ...,"Business,Leadership,Romance,Historical Romance...",875848419,3.73,7,The Human Equation: Building Profits by Puttin...,875848419 Why is common sense so uncommon when...


In [ ]:
loaded_books_df['tag'].to_csv(
    "./tagged_description.txt",
    sep="\n",
    index=False,
    header=False
)

In [ ]:
raw_document = TextLoader('./tagged_description.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_document)

Streaming output truncated to the last 5000 lines.


In [ ]:
print(documents[0])

page_content='002914180X Reveals that several hundred thousand Indians were affected by the Civil War and that twenty thousand Indians enlisted on both sides in an attempt to gain legitimacy, autonomy, or simply land.' metadata={'source': './tagged_description.txt'}


In [ ]:
len(documents)

69323

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
db_books = Chroma.from_documents(
    documents,
    embedding = OpenAIEmbeddings(),
)

In [ ]:
query = "a book to teach children about nature"
docs = db_books.similarity_search(query, k = 5)
docs

[Document(id='f5de3a2b-0b8b-4467-8be6-2cde323f5a6f', metadata={'source': './tagged_description.txt'}, page_content='746002750 -- Introduces young children to fundamental aspects of nature, science and technology-- Inspired by the questions children ask about the world around them,-- Simple text and detailed illustrations answer questions in clear, step-by-step stages'),
 Document(id='6f28668e-4ed7-4414-b6cb-2a86f94b1bc2', metadata={'source': './tagged_description.txt'}, page_content='1559715790 This fascinating series turns ordinary walks into adventures. Children learn to identify a variety of different plant, animal and insect species.,-- Helps children identify different species.,-- Includes scrapbook pages, for notes or drawings.,-- Features detailed true-to-life illustrations.'),
 Document(id='4cd003c0-c820-4276-9c24-bbe024d087c9', metadata={'source': './tagged_description.txt'}, page_content='076365549X This gorgeously illustrated volume of poetry â€” sprinkled with facts and fun

In [ ]:
loaded_books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75750 entries, 0 to 75749
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   author   75750 non-null  object 
 1   desc     75750 non-null  object 
 2   genre    69853 non-null  object 
 3   isbn     75750 non-null  object 
 4   rating   75750 non-null  float64
 5   reviews  75750 non-null  int64  
 6   title    75750 non-null  object 
 7   tag      75750 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 4.6+ MB


In [ ]:
def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 5
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 20)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [recs[i].page_content.strip('"').split()[0]]

    return loaded_books_df[loaded_books_df['isbn'].isin(books_list)].head(top_k)

In [ ]:
retrieve_semantic_recommendations(query)

,author,desc,genre,isbn,rating,reviews,title,tag
2443,"David M. Schwartz,Dwight Kuhn,Dwifht Kuhn","Look Once, Look Again, A Springboards into Sci...","Nature,Plants,Science",836825810,3.38,2,Plant Stems & Roots,"836825810 Look Once, Look Again, A Springboard..."
4318,Colleen Monroe,The Wonders of Nature Sketchbook gives aspirin...,"Art,Drawing",097549421X,4.20,2,The Wonders of Nature Sketchbook: Learn about ...,097549421X The Wonders of Nature Sketchbook gi...
12462,"Adam Gamble,Suwin Chan",Young children are invited to explore the wond...,"Childrens,Science,Geography",977797902,3.94,10,Good Night America,977797902 Young children are invited to explor...
15573,"Ken Ham,Mally Ham",An A-Z rhyme book with colorful cartoon illust...,"Childrens,Childrens,Picture Books,Nonfiction,C...",890511934,4.27,11,D is for Dinosaur,890511934 An A-Z rhyme book with colorful cart...
20174,Jen Green,Children learn that clean water is one of our ...,"Environment,Nonfiction,Childrens,Conservation,...",764131575,3.71,7,Why Should I Save Water?,764131575 Children learn that clean water is o...
